# Neural Network from Scratch — Public Dataset Exercise

This notebook adapts concepts from **Denny Britz’s** tutorial “Implementing a Neural Network from Scratch — An Introduction” 
and turns them into a hands‑on exercise using public datasets.

**Learning goals**
- Implement a 3‑layer neural network (NumPy only — no PyTorch/TensorFlow).
- Train on a **public dataset** (Digits or Iris) instead of random toy data.
- Practice **forward pass**, **loss**, and **backpropagation**.
- Evaluate accuracy and experiment with hyperparameters.

> Attribution: inspired by Denny Britz, *Implementing a Neural Network from Scratch in Python (2015)*.


## 0) Setup

Run the cell below to import the libraries we'll use. We’ll stick to NumPy and a couple of utility libraries.


In [ ]:
# Standard
import math, time, random
import numpy as np

# Data & utilities
from sklearn.datasets import load_digits, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Plotting (optional)
import matplotlib.pyplot as plt

np.random.seed(1337)

## 1) Choose your public dataset

We’ll avoid synthetic random blobs and use a small, public dataset that ships with scikit‑learn so no internet is required:

- **Digits (8×8 images, 10 classes)** — recommended (slightly larger, ~1797 samples).
- **Iris (4 features, 3 classes)** — very small/easy.

> 📌 You can switch by changing `DATASET` to `"digits"` or `"iris"` below.


In [ ]:
# TODO 1: Pick your dataset
DATASET = "digits"   # options: "digits" or "iris"

if DATASET == "digits":
    data = load_digits()
    X = data.data.astype(np.float64)        # shape: (n_samples, 64)
    y = data.target.astype(int)             # labels: 0..9
elif DATASET == "iris":
    data = load_iris()
    X = data.data.astype(np.float64)        # shape: (n_samples, 4)
    y = data.target.astype(int)             # labels: 0..2
else:
    raise ValueError("Unknown DATASET")

num_classes = len(np.unique(y))
print(f"Dataset: {DATASET}, X shape={X.shape}, classes={num_classes}")

## 2) Train/Val split and (optional) PCA for visualization

We’ll standardize features for faster training. For Digits (64‑D), we’ll also compute a 2‑D PCA projection for plotting.


In [ ]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1337)

# Standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)

# Optional: 2D PCA projection for visualization
pca2 = PCA(n_components=2, random_state=1337)
X_train_2d = pca2.fit_transform(X_train)
X_val_2d   = pca2.transform(X_val)

print("Train:", X_train.shape, "Val:", X_val.shape)

## 3) From‑scratch 3‑layer Neural Network (NumPy)

We’ll implement a simple fully‑connected network:
- Input → Hidden (ReLU) → Output (softmax)
- Loss: cross‑entropy
- Parameters initialized with small random values.

**Your tasks:**
- Implement the **forward** pass (`forward`).
- Compute the **loss** and **accuracy**.
- Implement **backpropagation** (`backward`) to get gradients for `W1, b1, W2, b2`.


In [ ]:
# TODO 2: Complete the forward and backward passes
def one_hot(y, num_classes):
    Y = np.zeros((y.size, num_classes), dtype=np.float64)
    Y[np.arange(y.size), y] = 1.0
    return Y

class MLP:
    def __init__(self, input_dim, hidden_dim, num_classes, weight_scale=0.01):
        # Parameters
        self.W1 = np.random.randn(input_dim, hidden_dim) * weight_scale
        self.b1 = np.zeros((1, hidden_dim), dtype=np.float64)
        self.W2 = np.random.randn(hidden_dim, num_classes) * weight_scale
        self.b2 = np.zeros((1, num_classes), dtype=np.float64)

        # caches for backward
        self.cache = {}

    @staticmethod
    def relu(z): 
        return np.maximum(0, z)

    @staticmethod
    def relu_grad(z):
        return (z > 0).astype(np.float64)

    def forward(self, X):
        """
        TODO 2a:
        - Compute z1 = X @ W1 + b1
        - a1 = ReLU(z1)
        - scores = a1 @ W2 + b2
        - softmax probabilities p = exp(scores)/sum(exp(scores), axis=1, keepdims=True)
        Return p, and cache intermediates needed for backward.
        """
        # ----- Your code here -----
        z1 = X @ self.W1 + self.b1
        a1 = self.relu(z1)
        scores = a1 @ self.W2 + self.b2
        # numerical stability
        scores -= scores.max(axis=1, keepdims=True)
        exp_scores = np.exp(scores)
        p = exp_scores / (exp_scores.sum(axis=1, keepdims=True) + 1e-12)
        # cache
        self.cache = {"X": X, "z1": z1, "a1": a1, "p": p}
        return p

    def loss(self, p, y):
        """Cross-entropy loss."""
        N = y.size
        # prevent log(0)
        log_likelihood = -np.log(p[np.arange(N), y] + 1e-12)
        return log_likelihood.mean()

    def backward(self, y):
        """
        TODO 2b: Backpropagate through softmax+CE, linear layers, and ReLU.
        Use cached values in self.cache. Compute grads for W2, b2, W1, b1.
        Return a dict with the gradients.
        """
        X  = self.cache["X"]
        z1 = self.cache["z1"]
        a1 = self.cache["a1"]
        p  = self.cache["p"]
        N  = y.size

        # gradient of loss wrt scores
        dscores = p.copy()
        dscores[np.arange(N), y] -= 1.0
        dscores /= N

        # grads for W2, b2
        dW2 = a1.T @ dscores
        db2 = dscores.sum(axis=0, keepdims=True)

        # backprop into a1
        da1 = dscores @ self.W2.T
        dz1 = da1 * self.relu_grad(z1)

        # grads for W1, b1
        dW1 = X.T @ dz1
        db1 = dz1.sum(axis=0, keepdims=True)

        return {"W1": dW1, "b1": db1, "W2": dW2, "b2": db2}

def accuracy(p, y):
    preds = p.argmax(axis=1)
    return (preds == y).mean()


## 4) Training loop (SGD)

We’ll train with plain stochastic gradient descent (mini‑batches) and an optional L2 regularizer.


In [ ]:
# Hyperparameters (feel free to tune)
hidden_dim = 64 if DATASET == "digits" else 16
lr = 0.1
weight_decay = 1e-4   # L2
epochs = 30
batch_size = 64

N, D = X_train.shape
mlp = MLP(input_dim=D, hidden_dim=hidden_dim, num_classes=num_classes)

Y_train_oh = one_hot(y_train, num_classes)

def iterate_minibatches(X, y, bs, shuffle=True):
    idx = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(idx)
    for i in range(0, X.shape[0], bs):
        j = idx[i:i+bs]
        yield X[j], y[j]

history = {"tr_loss": [], "tr_acc": [], "val_loss": [], "val_acc": []}

for epoch in range(1, epochs+1):
    # Train
    tr_loss_sum, tr_acc_sum, n_seen = 0.0, 0.0, 0
    for xb, yb in iterate_minibatches(X_train, y_train, batch_size, shuffle=True):
        # forward
        p = mlp.forward(xb)
        loss = mlp.loss(p, yb)
        # L2 regularization
        l2 = 0.5 * weight_decay * (np.sum(mlp.W1*mlp.W1) + np.sum(mlp.W2*mlp.W2))
        loss_total = loss + l2
        # backward
        grads = mlp.backward(yb)
        # add L2 grads
        grads["W1"] += weight_decay * mlp.W1
        grads["W2"] += weight_decay * mlp.W2
        # SGD update
        mlp.W1 -= lr * grads["W1"]
        mlp.b1 -= lr * grads["b1"]
        mlp.W2 -= lr * grads["W2"]
        mlp.b2 -= lr * grads["b2"]
        # stats
        tr_loss_sum += float(loss) * xb.shape[0]
        tr_acc_sum  += float(accuracy(p, yb)) * xb.shape[0]
        n_seen += xb.shape[0]

    # Evaluate
    p_val = mlp.forward(X_val)
    val_loss = mlp.loss(p_val, y_val)
    tr_loss = tr_loss_sum / n_seen
    tr_acc  = tr_acc_sum / n_seen
    val_acc = accuracy(p_val, y_val)

    history["tr_loss"].append(tr_loss)
    history["tr_acc"].append(tr_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    print(f"Epoch {epoch:02d} | train loss {tr_loss:.4f} acc {tr_acc:.3f} | val loss {val_loss:.4f} acc {val_acc:.3f}")

## 5) Plot training curves (optional)

In [ ]:
# Plot loss and accuracy over epochs
fig, ax = plt.subplots(1, 2, figsize=(10,4))
ax[0].plot(history['tr_loss'], label='train')
ax[0].plot(history['val_loss'], label='val')
ax[0].set_title('Loss'); ax[0].set_xlabel('epoch'); ax[0].legend()

ax[1].plot(history['tr_acc'], label='train')
ax[1].plot(history['val_acc'], label='val')
ax[1].set_title('Accuracy'); ax[1].set_xlabel('epoch'); ax[1].legend()
plt.show()

## 6) (Optional) 2D decision viz with PCA projection

For intuition only: we’ll plot the 2D PCA projection of the validation set colored by predicted class.


In [ ]:
# Only meaningful for a quick qualitative look
p_val = mlp.forward(X_val)
pred_val = p_val.argmax(axis=1)

plt.figure(figsize=(5,4))
plt.scatter(X_val_2d[:,0], X_val_2d[:,1], c=pred_val, s=20)
plt.title(f'Validation predictions (PCA‑2D) — {DATASET}')
plt.xlabel('PC1'); plt.ylabel('PC2'); plt.show()

## 7) Experiments (student prompts)

- **H1**: Increase `hidden_dim` and observe accuracy changes.
- **H2**: Try a smaller learning rate (`lr=0.05`), then larger (`lr=0.2`).
- **H3**: Remove weight decay (`weight_decay=0`) and compare overfitting.
- **H4**: Switch from `digits` to `iris` and adapt `epochs`/`hidden_dim`.
- **H5** *(challenge)*: Add another hidden layer.


---

## (Optional) Reference: Key equations

- **Forward**
  - $z_1 = XW_1 + b_1$
  - $a_1 = \max(0, z_1)$
  - $s = a_1 W_2 + b_2$
  - $p = \text{softmax}(s)$

- **Loss**
  - $\mathcal{L} = -\frac{1}{N}\sum_i \log p_{i,y_i}$

- **Backward (softmax+CE)**
  - $\frac{\partial \mathcal{L}}{\partial s} = (p - \text{one\_hot}(y))/N$

Then chain through linear and ReLU for $W_2, b_2, W_1, b_1$.
